In [4]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np

In [5]:
to_files = glob.glob("bylines/*-TO.csv")
from_files = glob.glob("bylines/*-FROM.csv")

In [6]:
def get_df(files,filenum):
    df = pd.read_csv(files[filenum])
    
    hours = []
    delay = []
    for index, row in df.iterrows():
        hours.append(row["scheduled_time"][:2])
        delay.append(int(row["delay_minutes"]))
        
    df["hour"] = hours
    df["delay"] = delay
    
    return df

In [7]:
def save_to_ny_df(df):
    save_to_ny = df[["week_day", "hour", "delay"]].copy()
    
    linename = str(df["line"].iloc[0]).replace(" ", "-")
    savefile = linename + "_TO-NY"
    
    return(save_to_ny)
    #save_to_ny.to_csv(r"forweb/{}.csv".format(savefile),index=False)

In [8]:
def save_from_ny_df(df):
    save_from_ny = df[["week_day", "hour", "delay"]].copy()
    
    linename = str(df["line"].iloc[0]).replace(" ", "-")
    savefile = linename + "_FROM-NY"
    
    save_from_ny.to_csv(r"forweb/{}.csv".format(savefile),index=False)

In [11]:
a = []
for filenum in range(len(to_files)):
    to_df = get_df(to_files,filenum)
    a.append(save_to_ny_df(to_df))

In [14]:
a[4].head()

,week_day,hour,delay
0,Wed,06,4
1,Wed,13,6
2,Wed,19,0
3,Wed,15,0
4,Wed,22,2


In [27]:
for filenum in range(len(from_files)):
    from_df = get_df(from_files,filenum)
    save_from_ny_df(from_df)

In [28]:
splitfiles = glob.glob("forweb/*-NY.csv")

In [86]:
for file in splitfiles:
    df = pd.read_csv(file)
    df_forhist = df[["delay"]].copy()
    
    df_forhist.to_csv(r"{}{}.csv".format(file[:-4],"-hist"),index=False)

In [46]:
for file1 in splitfiles:
    df1 = pd.read_csv(file1)
    m1 = df1.groupby(by=["week_day","hour"], as_index = False).mean()
    
    c1 = df1.groupby(by=["week_day","hour"])["delay"].apply(list)
    c1 = c1.reset_index()
    delay_pc = []
    for index1, row1 in c1.iterrows():
        zero_count = 0
        for ic in row1["delay"]:
            if ic == 0:
                zero_count = zero_count+1
        zeros = 1-zero_count/len(row1["delay"])
        delay_pc.append(zeros)
    c1["delay_percent"] = delay_pc
    
    heat_df = pd.merge(c1,m1, on=["week_day","hour"])
    heatdata = heat_df[["delay_percent","delay_y"]].values.tolist()
    heat_df["heatdata"] = heatdata
    
    df_forheat = heat_df[["week_day","hour","heatdata"]].copy()
    
    df_forheat.to_csv(r"{}{}.csv".format(file1,"-heat"),index=False)